# Detección de Intrusiones y Análisis de Anomalías en Tráfico de Red mediante Técnicas Estadísticas

**Universidad de La Habana, MATCOM**  
**Curso:** Estadística 2025-2026  
**Proyecto Final:** Análisis Estadístico Aplicado a Seguridad Informática

---

## 1. Introducción al Proyecto

### 1.1 Contexto y Motivación

La seguridad informática es uno de los pilares fundamentales en la infraestructura tecnológica moderna. Los Sistemas de Detección de Intrusiones (IDS) tradicionales, basados en firmas conocidas, presentan limitaciones significativas frente a ataques emergentes o modificados (zero-day attacks). 

Este proyecto propone un enfoque complementario basado en **análisis estadístico del comportamiento del tráfico de red**, permitiendo identificar patrones anómalos sin depender exclusivamente de firmas previamente catalogadas. Este tipo de aproximación resulta especialmente relevante en entornos dinámicos donde los ataques evolucionan constantemente.

### 1.2 Objetivos del Análisis

Este estudio busca responder **tres preguntas de investigación fundamentales**:

**Pregunta 1 (Análisis Comparativo):** ¿Existen diferencias estadísticamente significativas en el comportamiento de variables de flujo de red —como `src_bytes`, `dst_bytes` y `duration`— entre el tráfico normal y los distintos tipos de ataques (DoS, Probe, R2L y U2R)?

**Pregunta 2 (Reducción Dimensional):** ¿Es posible reducir la dimensionalidad de las 41 características del tráfico de red mediante Análisis de Componentes Principales (PCA), conservando al menos el 95% de la varianza explicada, y cómo impacta esta reducción en la visualización y separación de los distintos tipos de ataques?

**Pregunta 3 (Clasificación Comparativa):** ¿Qué técnica de clasificación estadística, Regresión Logística o K-Vecinos más Cercanos (K-NN), ofrece una mayor sensibilidad para detectar ataques raros (como U2R) en comparación con ataques volumétricos más comunes (como DoS)?

### 1.3 Dataset: NSL-KDD

**Fuente:** [NSL-KDD en Kaggle](https://www.kaggle.com/datasets/hassan06/nslkdd)

El dataset NSL-KDD es una versión refinada del clásico KDD Cup 1999, diseñada específicamente para eliminar redundancias y sesgos presentes en el conjunto original. Es ampliamente reconocido como estándar académico para la evaluación de algoritmos de detección de intrusiones.

**Características principales:**
- **41 variables predictoras** + 1 variable objetivo (`attack_type`) + 1 nivel de dificultad
- **Tipos de ataques:** Normal, DoS (Denial of Service), Probe (escaneo/sondeo), R2L (Remote to Local), U2R (User to Root)
- **Conjunto de entrenamiento:** 25,192 observaciones
- **Conjunto de prueba:** 22,544 observaciones

**Categorización de variables:**
- **Básicas:** Derivadas de cabeceras TCP/IP (duration, protocol_type, src_bytes, dst_bytes, flag)
- **De contenido:** Información sobre el payload (num_failed_logins, root_shell, etc.)
- **De tráfico:** Estadísticas temporales orientadas a detectar patrones (count, serror_rate, etc.)

---

## 1.4 Carga de Datos y Preparación Inicial

# Imports y Configuración

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configuración de visualizaciones
warnings.filterwarnings('ignore')
%matplotlib inline

# Estilo global para mantener consistencia en todas las visualizaciones
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

# Paleta de colores consistente para categorías de ataque
# Se utilizará en todas las visualizaciones del proyecto
attack_colors = {
    'Normal': '#2ecc71',    # Verde - Tráfico legítimo
    'DoS': '#e74c3c',       # Rojo - Ataques de denegación de servicio
    'Probe': '#f39c12',     # Naranja - Ataques de reconocimiento
    'R2L': '#9b59b6',       # Morado - Acceso remoto no autorizado
    'U2R': '#34495e'        # Gris oscuro - Escalada de privilegios
}

print("✅ Librerías importadas y configuración de visualización establecida")

## Carga de Datos

In [ ]:
# Definir los nombres de las columnas (43 columnas en total)
col_names = [
    "duration", "protocol_type", "service", "flag", "src_bytes",
    "dst_bytes", "land", "wrong_fragment", "urgent", "hot",
    "num_failed_logins", "logged_in", "num_compromised", "root_shell",
    "su_attempted", "num_root", "num_file_creations", "num_shells",
    "num_access_files", "num_outbound_cmds", "is_host_login",
    "is_guest_login", "count", "srv_count", "serror_rate",
    "srv_serror_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate",
    "diff_srv_rate", "srv_diff_host_rate", "dst_host_count",
    "dst_host_srv_count", "dst_host_same_srv_rate",
    "dst_host_diff_srv_rate", "dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate", "dst_host_serror_rate",
    "dst_host_srv_serror_rate", "dst_host_rerror_rate",
    "dst_host_srv_rerror_rate", "attack_type", "difficulty_level"
]

# Cargar los datasets
# Nota: Ajusta las rutas según tu estructura de carpetas
train_df = pd.read_csv('Data/KDDTrain+_20Percent.txt', 
                       names=col_names, 
                       header=None)

test_df = pd.read_csv('Data/KDDTest+.txt', 
                      names=col_names, 
                      header=None)

# Crear variable binaria para clasificación binaria (Normal vs. Ataque)
train_df['is_attack'] = (train_df['attack_type'] != 'normal').astype(int)
test_df['is_attack'] = (test_df['attack_type'] != 'normal').astype(int)

# Mostrar información básica
print(f"📊 Datos de entrenamiento: {train_df.shape}")
print(f"📊 Datos de prueba: {test_df.shape}")
print(f"\n✅ Datasets cargados exitosamente")

# Distribución binaria inicial
print(f"\n🎯 Distribución binaria en entrenamiento (Normal vs. Ataque):")
print(train_df['is_attack'].value_counts(normalize=True).round(4))

print(f"\n🎯 Distribución binaria en prueba (Normal vs. Ataque):")
print(test_df['is_attack'].value_counts(normalize=True).round(4))